In [1]:
from openai import OpenAI

import minsearch
import json

from elasticsearch import Elasticsearch
from elasticsearch import NotFoundError

import requests 

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Elastic Search

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
try:
    es_client.indices.get(index=index_name)
    print(f"{index_name} already exists")
except NotFoundError:
    es_client.indices.create(index=index_name, body=index_settings)
    


course-questions already exists


In [6]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4","text"],
                        #"fields": [],
                        #"type": "best_fields"
                    }
                },
                 "filter": {
                     "term": {
                         "course": "machine-learning-zoomcamp"
                     }
                 }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        #print(response['hits'])
        result_docs.append(hit['_source'])
    
    return result_docs

In [7]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████| 948/948 [00:21<00:00, 43.10it/s]


In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    
    context_template = ""

    for doc in search_results:
         context_template +=  f"Q: {doc['question']}\nA: {doc['text']}\n\n" 

    
    final_prompt = prompt_template.format(question=query, context=context_template).strip()
    return final_prompt

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'phi3',
        messages = [{"role":"user", "content":prompt}]
    )
    return response.choices[0].message.content

In [11]:
query = "How do I execute a command in a running docker container?" 

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return prompt, search_results, answer

In [ ]:
prompt, search_results, answer = rag(query)

In [ ]:
answer

In [ ]:
prompt

In [ ]:
len(prompt)

In [ ]:
search_results

In [ ]:
query = "How do I execute a command in a running docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context_template = """
Q: {question}
A: {text}
""".strip()


context_entries = "\n\n".join([context_template.format(question=doc['question'], text=doc['text']) for doc in search_results])

prompt = prompt_template.format(question=query, context=context_entries).strip()
print(prompt)
print(len(prompt))


Tiktoken

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
#encoding.encode(prompt)

In [ ]:
len(encoding.encode(prompt))

In [ ]:
encoding.decode_single_token_bytes(290)